In [1]:
#import tensorflow as tf
import datetime
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import TensorBoard
import os, shutil
from PIL import Image, ImageDraw, ImageFont, ImageColor
import random
import pandas as pd

In [2]:
%reload_ext tensorboard

In [3]:
train_dataset_dir ='\\Users\\danie\\Penn_State_REU_Jupyter\\Hexagon ML Project\\Training_set_6_15' 

df=pd.read_csv(os.path.join(train_dataset_dir,"training_dataframe.csv"))
df = df.sample(frac=1).reset_index(drop=True)

In [5]:
df.shape

(121500, 9)

In [8]:
def build_model():
    model = models.Sequential()                                  
    model.add(layers.Conv2D(16, (3,3) ,activation='relu', input_shape=(32,32,3)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(16,(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(8))
    return model

In [9]:
#Creates generators for training and validation images
SEED=5
validation_split_frac=0.2
batch_size=256
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=validation_split_frac) 
train_generator= train_datagen.flow_from_dataframe(
        df,
        directory=train_dataset_dir,
        x_col="filename",
        y_col=['x_coord','y_coord','neighbor1x','neighbor1y','neighbor2x','neighbor2y','neighbor3x','neighbor3y'],
        target_size=(32,32),
        shuffle=True,
        seed= SEED,
        class_mode="raw",
        batch_size=batch_size,
        subset='training')

validation_generator= train_datagen.flow_from_dataframe(
        df,
        directory=train_dataset_dir,
        x_col="filename",
        y_col=['x_coord','y_coord','neighbor1x','neighbor1y','neighbor2x','neighbor2y','neighbor3x','neighbor3y'],
        target_size=(32,32),
        shuffle=True,
        seed= SEED,
        class_mode="raw",
        batch_size=batch_size,
        subset='validation')

Found 97200 validated image filenames.
Found 24300 validated image filenames.


In [11]:
model=build_model()
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])
#model.summary()
num_epochs=20
training_steps=int(df.shape[0]*(1-validation_split_frac)/batch_size)
validation_steps=int(df.shape[0]*validation_split_frac/batch_size)
#assert (training_steps+validation_steps)*batch_size == df.shape[0]
log_dir = os.path.join("logs\\fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(
    train_generator,
    steps_per_epoch=training_steps,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=num_epochs,
    verbose=1, 
    callbacks=[tensorboard_callback]
    )


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 379 steps, validate for 94 steps
Epoch 1/20
379/379 [==============================] - 74s 194ms/step - loss: 0.0345 - mae: 0.1569 - val_loss: 0.0271 - val_mae: 0.1406
Epoch 2/20
379/379 [==============================] - 58s 152ms/step - loss: 0.0268 - mae: 0.1395 - val_loss: 0.0262 - val_mae: 0.1381
Epoch 3/20
379/379 [==============================] - 57s 151ms/step - loss: 0.0258 - mae: 0.1367 - val_loss: 0.0252 - val_mae: 0.1352
Epoch 4/20
379/379 [==============================] - 55s 145ms/step - loss: 0.0248 - mae: 0.1342 - val_loss: 0.0243 - val_mae: 0.1326
Epoch 5/20
379/379 [==============================] - 56s 149ms/step - loss: 0.0238 - mae: 0.1309 - val_loss: 0.0231 - val_mae: 0.1288
Epoch 6/20
379/379 [==============================] - 59s 156ms/step - loss: 0.0226 - mae: 0.1269 - val_loss: 0.0220 - val_mae: 0.1245
Epoch 7/20
379/379 [==============================] - 54s 143ms/step - loss: 0.0214 - mae: 0.1224

In [9]:
model=build_model()
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])
#model.summary()

In [13]:
from tensorboard import notebook
notebook.list()
%tensorboard --logdir /logs/fit --port 0

Known TensorBoard instances:
  - port 6006: logdir logs/fit (started 5 days, 20:26:52 ago; pid 10216)
  - port 6006: logdir logs/fit (started 5 days, 0:48:56 ago; pid 10656)
  - port 6006: logdir Training_set_6_10/logs/fit (started 4 days, 20:04:09 ago; pid 13120)
  - port 6006: logdir Training_set_6_10/logs/fit (started 4 days, 19:58:34 ago; pid 15348)
  - port 6006: logdir logs (started 4 days, 20:27:04 ago; pid 16588)


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 24248.

UsageError: Line magic function `%jobs` not found.


In [ ]:

#Loads labels and training data into two arrays, such that their indicies match
# train_labels=[]
# with open("Training_set_6_10\\training_labels.json",'r') as dict_contents:
#     train_labels_dict=json.load(dict_contents)
# train_labels= [value for key,value in sorted(train_labels_dict.items())]

# train_images1 = []
# path = original_dataset_dir
# valid_images = [".jpg",".gif",".png",".tga"]
# for f in os.listdir(path):
#     ext = os.path.splitext(f)[1]
#     if ext.lower() not in valid_images:
#         continue
#     #print(f[10:-4])
#     train_images1.append(Image.open(os.path.join(path,f)))
#     train_targets1.append(train_targets_dict[str(f[10:-4])])


In [ ]:
# k = 4
# num_val_samples = len(train_images) // k
# num_epochs = 20
# all_histories = []
# train_datagen = ImageDataGenerator(rescale=1./255)             
# val_datagen = ImageDataGenerator(rescale=1./255) 
# for i in range(k):
#     print('processing fold #', i)
#     val_images = train_images[i * num_val_samples: (i + 1) * num_val_samples]    
#     val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

#     partial_train_images = train_images[:i * num_val_samples] + train_images[(i + 1) * num_val_samples:]
#     partial_train_targets = train_targets[:i * num_val_samples] + train_targets[(i + 1) * num_val_samples:]
    
#     train_generator= train_datagen.flow(
#         partial_train_images,
#         y=partial_train_targets,
#         shuffle=True,
#         batch_size=1)
#     val_generator= val_datagen.flow(
#         val_images,
#         y=val_targets,
#         shuffle=True,
#         batch_size=1)

#     model = build_model()                                                    
#     history= model.fit_generator(
#         train_generator,
#         steps_per_epoch=len(partial_train_images),
#         epochs=num_epochs,
#         validation_data=val_generator,
#         validation_steps=num_val_samples)
#     #mae_history = history.history['val_mean_absolute_error']
#     all_histories.append(history)

In [ ]:
# #index_list=random.sample(range(len(train_images)),len(train_images))
# index_list=[12, 77, 11, 32, 35, 28, 58, 70, 0, 46, 64, 74, 13, 1, 19, 2, 57, 39, 56, 30, 71, 49, 38, 7, 47, 75, 43, 25, 5, 44, 18, 50, 51, 17, 21, 27, 48, 29, 6, 10, 8, 34, 76, 9, 69, 60, 62, 15, 65, 53, 59, 16, 24, 4, 55, 22, 63, 78, 73, 67, 20, 42, 68, 52, 26, 79, 36, 33, 45, 41, 37, 72, 14, 40, 66, 3, 54, 31, 61, 23]
# train_images=[]
# train_targets=[]
# for i in index_list:
#     train_images.append(train_images1[i])
#     train_targets.append(train_targets1[i])


In [ ]:
# fig = plt.figure(figsize = (10, 10))
# ax = plt.subplot(1,1,1)
# ax.imshow(train_images1[3])
# ax.axis('off')